In [8]:
import os
import paramiko

# connect with ssh
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('10.1.1.124', username='hadoop', password='ubuntu')

First analyze the letter frequencies in books across the eras. As performance is not relevant now, the job will be executed with 1 reducer, using combiner.
We hypotize that the input files are already in the input folder in hdfs, in /user/hadoop/letter_analysis/input/.

In [9]:
root_dir = '/user/hadoop/historical_analysis'
jar_name = 'letter-counter-1.0-SNAPSHOT.jar'
input_dir = os.path.join(root_dir, 'input')
output_dir = os.path.join(root_dir, 'output')
# remove old outputs
ssh.exec_command('hadoop fs -rm -r ' + output_dir)

# run the job for each file
for txt_file in os.listdir('../resources/historical_analysis'):
    ssh.exec_command('hadoop jar ' + jar_name + ' ' +
                input_dir + '/' + txt_file + ' ' + 
                output_dir + ' ' +
                '1 combiner' # number of reducers and method to use
              )
    # copy the output to local
    os.system('hadoop fs -copyToLocal ' + output_dir + '/' + txt_file + ' ../resources/historical_analysis/output')

ssh.close()

After that we can analyze the output files.